In [1]:
import subprocess
import sys

import overpy
import pandas as pd
from shapely.geometry import Polygon

sys.path.append("/home/ppeter/repo/mcr-py/src/")
from package.geometa import GeoMeta

In [2]:
gtfs_key = 781
city_name_german = "Berlin"
city_name = "Berlin"
center = (52.5170365, 13.3888599)
start_date = "2024-05-27"
end_date = "2024-06-02"
bbox = {
    "max_lat": 52.5170365 + 0.05,
    "min_lat": 52.5170365 - 0.05,
    "max_lon": 13.3888599 + 0.04,
    "min_lon": 13.3888599 - 0.07,
}

In [3]:
geometa_path = f"../data/stateful_variables/{city_name.lower()}_geometa.pkl"
today = pd.Timestamp.today().strftime("%Y%m%d")

In [4]:
subprocess.run(
    [
        "python",
        "../src/main.py",
        "gtfs",
        "download",
        str(gtfs_key),
        f"../data/gtfs-raw/{city_name.lower()}_{today}.zip",
    ]
)

[13:14:44] INFO     Downloading GTFS feed with ID 781             ]8;id=242752;file:///home/ppeter/repo/mcr-py/src/package/gtfs/catalog.py\catalog.py]8;;\:]8;id=324474;file:///home/ppeter/repo/mcr-py/src/package/gtfs/catalog.py#155\155]8;;\
[13:14:56] INFO     Downloading GTFS feed with ID 781 done (12.75 ]8;id=236405;file:///home/ppeter/repo/mcr-py/src/package/gtfs/catalog.py\catalog.py]8;;\:]8;id=600046;file:///home/ppeter/repo/mcr-py/src/package/gtfs/catalog.py#155\155]8;;\
                    seconds)                                                    


CompletedProcess(args=['python', '../src/main.py', 'gtfs', 'download', '781', '../data/gtfs-raw/berlin_20240507.zip'], returncode=0)

In [5]:
polygon = Polygon(
    [
        [bbox["min_lon"], bbox["min_lat"]],
        [bbox["min_lon"], bbox["max_lat"]],
        [bbox["max_lon"], bbox["max_lat"]],
        [bbox["max_lon"], bbox["min_lat"]],
    ]
)

geometa = GeoMeta(boundary=polygon)
geometa.save(geometa_path)

In [6]:
def order_ways_and_nodes(result):
    ways_dict = {way.id: [node for node in way.nodes] for way in result.ways}

    current_way_id, current_way_nodes = ways_dict.popitem()
    ordered_nodes = current_way_nodes

    while ways_dict:
        previous_way_id = current_way_id
        for next_way_id, next_way_nodes in ways_dict.items():
            if ordered_nodes[-1] == next_way_nodes[0]:
                ordered_nodes.extend(next_way_nodes[1:])
                current_way_id = next_way_id
                break
            elif ordered_nodes[-1] == next_way_nodes[-1]:
                ordered_nodes.extend(reversed(next_way_nodes[:-1]))
                current_way_id = next_way_id
                break
        if previous_way_id == current_way_id:
            break
        ways_dict.pop(current_way_id)

    return [(node.lat, node.lon) for node in ordered_nodes]


def fetch_boundary_coords(city_name):
    api = overpy.Overpass()

    # Koeln -> Admin level 6
    # Berlin -> Admin level 4
    query = f"""
    [out:json][timeout:50];
    area["name"="{city_name}"]->.searchArea;
    relation["boundary"="administrative"]["admin_level"="4"](area.searchArea);
    out body;
    >;
    out skel qt;
    """

    result = api.query(query)
    boundary_coords = order_ways_and_nodes(result)

    return boundary_coords


boundary_coords = fetch_boundary_coords(city_name_german)
polygon = Polygon([(lon, lat) for lat, lon in boundary_coords])
geometa = GeoMeta(boundary=polygon)
geometa.save(geometa_path)

In [7]:
for start_date_ in pd.date_range(start_date, end_date, freq="1d"):
    end_date_ = start_date_ + pd.Timedelta(days=1)
    subprocess.run(
        [
            "python",
            "../src/main.py",
            "gtfs",
            "crop",
            f"../data/gtfs-raw/{city_name.lower()}_{today}.zip",
            f"../data/gtfs-reduced/{city_name.lower()}_gtfs_{start_date_.strftime('%Y%m%d')}.zip",
            "--geometa-path",
            geometa_path,
            "--time-start",
            start_date_.strftime("%d.%m.%Y-00:00:00"),
            "--time-end",
            end_date_.strftime("%d.%m.%Y-00:00:00"),
        ]
    )
    subprocess.run(
        [
            "python",
            "../src/main.py",
            "gtfs",
            "clean",
            f"../data/gtfs-reduced/{city_name.lower()}_gtfs_{start_date_.strftime('%Y%m%d')}.zip",
            f"../data/gtfs-cleaned/{city_name.lower()}_{start_date_.strftime('%Y%m%d')}",
        ]
    )
    subprocess.run(
        [
            "python",
            "../src/main.py",
            "build-structures",
            f"../data/gtfs-cleaned/{city_name.lower()}_{start_date_.strftime('%Y%m%d')}",
            f"../data/gtfs-cleaned/{city_name.lower()}_{start_date_.strftime('%Y%m%d')}/structs.pkl",
        ]
    )

[13:14:59] INFO     Cropping GTFS data                               ]8;id=229424;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py\gtfs.py]8;;\:]8;id=225645;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py#139\139]8;;\
           INFO     Reading GTFS data                                 ]8;id=875830;file:///home/ppeter/repo/mcr-py/src/package/gtfs/crop.py\crop.py]8;;\:]8;id=945065;file:///home/ppeter/repo/mcr-py/src/package/gtfs/crop.py#19\19]8;;\


/home/ppeter/repo/mcr-py/src/package/gtfs/archive.py:59: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, dtype=dtypes.GTFS_DTYPES)  # type: ignore


[13:15:04] INFO     Reading GTFS data done (5.09 seconds)             ]8;id=424036;file:///home/ppeter/repo/mcr-py/src/package/gtfs/crop.py\crop.py]8;;\:]8;id=826055;file:///home/ppeter/repo/mcr-py/src/package/gtfs/crop.py#19\19]8;;\
[13:15:08] INFO             Crop results:                            ]8;id=206367;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py\gtfs.py]8;;\:]8;id=991088;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py#139\139]8;;\
                            # of trips: 224212 (86.10%)                         
                            # of stop times: 5260481 (87.66%)                   
                            # of stops: 9799 (23.62%)                           
[13:15:17] INFO     Cropping GTFS data done (18.63 seconds)          ]8;id=761190;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py\gtfs.py]8;;\:]8;id=205693;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py#139\139]8;;\
[13:15:19] INFO     Cleaning 

/home/ppeter/repo/mcr-py/src/package/gtfs/archive.py:59: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, dtype=dtypes.GTFS_DTYPES)  # type: ignore


[13:18:46] INFO     Reading GTFS data done (5.02 seconds)             ]8;id=883099;file:///home/ppeter/repo/mcr-py/src/package/gtfs/crop.py\crop.py]8;;\:]8;id=489796;file:///home/ppeter/repo/mcr-py/src/package/gtfs/crop.py#19\19]8;;\
[13:18:49] INFO             Crop results:                            ]8;id=378565;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py\gtfs.py]8;;\:]8;id=514498;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py#139\139]8;;\
                            # of trips: 224212 (86.10%)                         
                            # of stop times: 5260481 (87.66%)                   
                            # of stops: 9799 (23.62%)                           
[13:18:59] INFO     Cropping GTFS data done (18.67 seconds)          ]8;id=817590;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py\gtfs.py]8;;\:]8;id=200095;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py#139\139]8;;\
[13:19:01] INFO     Cleaning 

...

[13:29:48] INFO     Reading GTFS data done (5.00 seconds)             ]8;id=436980;file:///home/ppeter/repo/mcr-py/src/package/gtfs/crop.py\crop.py]8;;\:]8;id=600950;file:///home/ppeter/repo/mcr-py/src/package/gtfs/crop.py#19\19]8;;\
[13:29:51] INFO             Crop results:                            ]8;id=294101;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py\gtfs.py]8;;\:]8;id=10626;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py#139\139]8;;\
                            # of trips: 224212 (86.10%)                         
                            # of stop times: 5260481 (87.66%)                   
                            # of stops: 9799 (23.62%)                           
[13:30:01] INFO     Cropping GTFS data done (18.57 seconds)          ]8;id=765046;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py\gtfs.py]8;;\:]8;id=923464;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py#139\139]8;;\
[13:30:03] INFO     Cleaning G

/home/ppeter/repo/mcr-py/src/package/gtfs/archive.py:59: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, dtype=dtypes.GTFS_DTYPES)  # type: ignore


[13:33:29] INFO     Reading GTFS data done (5.02 seconds)             ]8;id=325379;file:///home/ppeter/repo/mcr-py/src/package/gtfs/crop.py\crop.py]8;;\:]8;id=472158;file:///home/ppeter/repo/mcr-py/src/package/gtfs/crop.py#19\19]8;;\
[13:33:33] INFO             Crop results:                            ]8;id=752823;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py\gtfs.py]8;;\:]8;id=378843;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py#139\139]8;;\
                            # of trips: 224212 (86.10%)                         
                            # of stop times: 5260481 (87.66%)                   
                            # of stops: 9799 (23.62%)                           
[13:33:43] INFO     Cropping GTFS data done (18.64 seconds)          ]8;id=205742;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py\gtfs.py]8;;\:]8;id=134365;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py#139\139]8;;\
[13:33:45] INFO     Cleaning 

/home/ppeter/repo/mcr-py/src/package/gtfs/archive.py:59: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, dtype=dtypes.GTFS_DTYPES)  # type: ignore


[13:37:11] INFO     Reading GTFS data done (4.92 seconds)             ]8;id=649728;file:///home/ppeter/repo/mcr-py/src/package/gtfs/crop.py\crop.py]8;;\:]8;id=629258;file:///home/ppeter/repo/mcr-py/src/package/gtfs/crop.py#19\19]8;;\
[13:37:14] INFO             Crop results:                            ]8;id=11719;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py\gtfs.py]8;;\:]8;id=258388;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py#139\139]8;;\
                            # of trips: 224212 (86.10%)                         
                            # of stop times: 5260481 (87.66%)                   
                            # of stops: 9799 (23.62%)                           
[13:37:24] INFO     Cropping GTFS data done (18.32 seconds)          ]8;id=648789;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py\gtfs.py]8;;\:]8;id=934760;file:///home/ppeter/repo/mcr-py/src/command/gtfs/gtfs.py#139\139]8;;\
[13:37:26] INFO     Cleaning G